<a href="https://colab.research.google.com/github/tavares-fiap/cp2-datasciense/blob/main/cp2_ds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Instalação de bibliotecas necessárias
!pip install jinja2
!pip install matplotlib seaborn pandas

In [11]:
import pandas as pd

# Carregar o dataset enviado pelo usuário
file_path = '/content/MarvelVsDC.csv'
df = pd.read_csv(file_path)

# Exibir as primeiras linhas do dataset para verificar sua estrutura
df.head()


,ID,Movie,Year,Genre,RunTime,Description,IMDB_Score
0,0,Eternals,-2021,"Action,Adventure,Drama",0,"The saga of the Eternals, a race of immortal b...",0.0
1,1,Loki,(2021– ),"Action,Adventure,Fantasy",0,A new Marvel chapter with Loki at its center.,0.0
2,2,The Falcon and the Winter Soldier,-2021,"Action,Adventure,Drama",50 min,"Following the events of 'Avengers: Endgame,' S...",7.5
3,3,WandaVision,-2021,"Action,Comedy,Drama",350 min,Blends the style of classic sitcoms with the M...,8.1
4,4,Spider-Man: No Way Home,-2021,"Action,Adventure,Sci-Fi",0,A continuation of Spider-Man: Far From Home.,0.0


In [21]:
import matplotlib.pyplot as plt
import seaborn as sns
from jinja2 import Environment, FileSystemLoader
import os
import base64
from io import BytesIO
import numpy as np

# Definindo o template HTML para o relatório
template_html = """
<!DOCTYPE html>
<html lang="pt-br">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Relatório com Análise dos Dados de Filmes - Marvel vs DC</title>
    <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
</head>
<body class="bg-light">
    <div class="container my-5">
        <h1 class="text-center mb-4">Relatório com Análise dos Dados de Filmes - Marvel vs DC</h1>
        <div class="card mb-4">
            <div class="card-body">
                <h2 class="card-title">Sumário</h2>
                <p><strong>Número de Linhas:</strong> {{ summary.shape[0] }}</p>
                <p><strong>Número de Colunas:</strong> {{ summary.shape[1] }}</p>
                <p><strong>Células Faltantes:</strong> {{ summary.cels_faltantes }}</p>
                <p><strong>Linhas Duplicadas:</strong> {{ summary.linhas_duplicadas }}</p>
            </div>
        </div>

        <div class="card mb-4">
            <div class="card-body">
                <h3 class="card-title">Tipos de Data</h3>
                <ul class="list-group">
                    {% for column, dtype in summary.dtypes.items() %}
                    <li class="list-group-item">{{ column }}: {{ dtype }}</li>
                    {% endfor %}
                </ul>
            </div>
        </div>

        <div class="card mb-4">
            <div class="card-body">
                <h3 class="card-title">Valores Faltantes</h3>
                <ul class="list-group">
                    {% for column, missing in summary.valores_faltantes.items() %}
                    <li class="list-group-item">{{ column }}: {{ missing }}</li>
                    {% endfor %}
                </ul>
            </div>
        </div>

        <h2 class="mt-5">Gráficos</h2>
        <div class="row">
            {% for plot in plots %}
            <div class="col-md-6 mb-4">
                <div class="card">
                    <img src="data:image/png;base64,{{ plot }}" class="card-img-top" alt="Plot">
                </div>
            </div>
            {% endfor %}
        </div>
    </div>

    <script src="https://code.jquery.com/jquery-3.5.1.slim.min.js"></script>
    <script src="https://cdn.jsdelivr.net/npm/@popperjs/core@2.9.2/dist/umd/popper.min.js"></script>
    <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/js/bootstrap.min.js"></script>
</body>
</html>
"""

# Salvando o template HTML no sistema de arquivos
template_dir = '/content/templates'
os.makedirs(template_dir, exist_ok=True)
with open(os.path.join(template_dir, 'report_template.html'), 'w') as file:
    file.write(template_html)

# Função para gerar resumo do dataframe
def resumo_df(df):
    num_var = len(df.columns)
    num_obs = len(df)
    cels_faltantes = df.isnull().sum().sum()
    linhas_duplicadas = df.duplicated().sum()

    summary = {
        'shape': df.shape,
        'num_var': num_var,
        'num_obs': num_obs,
        'cels_faltantes': cels_faltantes,
        'linhas_duplicadas': linhas_duplicadas,
        'dtypes': df.dtypes.to_dict(),
        'valores_faltantes': df.isnull().sum().to_dict(),
    }
    return summary

# Função para converter gráficos para Base64
def plot_64(fig):
    buf = BytesIO()
    fig.savefig(buf, format='png')
    buf.seek(0)
    img_base64 = base64.b64encode(buf.read()).decode('utf-8')
    buf.close()
    return img_base64

# Função para gerar gráficos e retornar como Base64
# Função para gerar gráficos com ajustes
def criar_plot(df):
    plots = []

    # Verificação e conversão de colunas para garantir que sejam do tipo correto
    for column in df.columns:
        if df[column].dtype == 'object':
            try:
                df[column] = pd.to_numeric(df[column])
            except ValueError:
                pass  # Se não for possível converter, mantenha como string

    # Histogramas para variáveis numéricas
    for column in df.select_dtypes(include=np.number).columns:
        fig, ax = plt.subplots()
        sns.histplot(df[column], kde=True, ax=ax)
        plt.title(f'Histograma de {column}')
        img_base64 = plot_64(fig)
        plots.append(img_base64)
        plt.close(fig)

    # Gráficos de contagem para variáveis categóricas
    for column in df.select_dtypes(include='object').columns:
        if column == 'Description':
            continue  # Ignorar a coluna 'Description'

        fig, ax = plt.subplots(figsize=(20, 18))  # Aumenta o tamanho do gráfico
        sns.countplot(y=df[column], ax=ax)
        plt.title(f'Countplot de {column}')

        # Ajuste a rotação dos rótulos e o tamanho da fonte
        ax.set_yticklabels(ax.get_yticklabels(), fontsize=8, rotation=0, ha='right')  # Ajuste o tamanho e a rotação

        # Ajuste automático de layout para evitar sobreposição
        plt.tight_layout()

        img_base64 = plot_64(fig)
        plots.append(img_base64)
        plt.close(fig)

    # Gráfico de barras empilhadas: Comparação Marvel vs DC
    if 'Universe' in df.columns:
        fig, ax = plt.subplots(figsize=(20, 18))
        sns.countplot(data=df, x='Universe', hue='Type', ax=ax)
        plt.title('Marvel vs DC por Tipo (Filme/Série)')
        plt.tight_layout()
        img_base64 = plot_64(fig)
        plots.append(img_base64)
        plt.close(fig)

    # Boxplot: Comparação de IMDb por Universo (Marvel vs DC)
    if 'Universe' in df.columns and 'IMDb' in df.columns:
        fig, ax = plt.subplots(figsize=(20, 18))
        sns.boxplot(data=df, x='Universe', y='IMDb', ax=ax)
        plt.title('Distribuição das Notas IMDb por Universo')
        plt.tight_layout()
        img_base64 = plot_64(fig)
        plots.append(img_base64)
        plt.close(fig)

    # Scatter plot: Nota IMDb em função do ano de lançamento
    if 'IMDb' in df.columns and 'Year' in df.columns:
        fig, ax = plt.subplots(figsize=(20, 18))
        sns.scatterplot(data=df, x='Year', y='IMDb', hue='Universe', ax=ax)
        plt.title('IMDb por Ano de Lançamento (Marvel vs DC)')
        plt.tight_layout()
        img_base64 = plot_64(fig)
        plots.append(img_base64)
        plt.close(fig)

    return plots


# Função para gerar relatório HTML
def gerar_html(df):
    env = Environment(loader=FileSystemLoader(template_dir))
    template = env.get_template('report_template.html')

    summary = resumo_df(df)
    plots = criar_plot(df)

    html_content = template.render(summary=summary, plots=plots)

    # Salvar o relatório HTML
    report_path = '/content/relatorio_marvel_dc.html'
    with open(report_path, 'w') as f:
        f.write(html_content)

    print(f'Report saved at: {report_path}')
    return report_path

# Gerar o relatório HTML
report_path = gerar_html(df)
report_path


<ipython-input-21-acbe820ec512>:138: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(ax.get_yticklabels(), fontsize=8, rotation=0, ha='right')  # Ajuste o tamanho e a rotação
<ipython-input-21-acbe820ec512>:138: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(ax.get_yticklabels(), fontsize=8, rotation=0, ha='right')  # Ajuste o tamanho e a rotação
<ipython-input-21-acbe820ec512>:138: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(ax.get_yticklabels(), fontsize=8, rotation=0, ha='right')  # Ajuste o tamanho e a rotação
<ipython-input-21-acbe820ec512>:138: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(ax.get_yticklabels(), fontsize=8, rotation=0, ha='right')  # Ajuste o tamanho e a rotação


Report saved at: /content/relatorio_marvel_dc.html


'/content/relatorio_marvel_dc.html'